In [12]:
import requests
import socket
import json
import numpy as np
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift
import math
import h5py
from requests.adapters import HTTPAdapter
from urllib3.connection import HTTPConnection
import datetime as dt
from sigmf import SigMFFile
from sigmf.utils import get_data_type_str
import os 
import psutil

In [3]:
#Dataset Generator for creating datasets
#Creates IQ datasets within the SigMF format
#IQ files as well as signal metadata
class sigMFDataset():
    def __init__(self):
        self.date_time = dt.datetime.utcnow().isoformat()+'Z'
        self.metadataIsSet = False
        
    def setData(self,data,label,samplesPerExample):
        self.data = data
        self.label = label
        self.SPE = samplesPerExample
        
    def createDataset(self):
        if self.metadataIsSet:
            self.createFolder()
            self.createIQFile()
            self.createMetadata()
        else:
            print("Set metadata first with setMetadata()")
    
    def createFolder(self):
        parent_dir = os.getcwd()
        directory = self.fileName+"_"+self.author+"_"+self.date_time
        self.path = os.path.join(parent_dir,directory)
        os.mkdir(self.path) 
        print("Directory '% s' created" % directory) 
    
    def createIQFile(self):
        self.data.tofile(self.fileName+'.sigmf-data')
    
    def setMetadata(self):
        # create the metadata
        self.fileName = input("File Name:")
        self.samp_rate = input("Sampling Rate:")
        self.freq = input("Sampling Frequency:")
        self.author = input("Author Email:")
        self.description = input("Description:")
        self.metadataIsSet = True
        
    def setMetadata(self,fileName,samp_rate,freq,author,description):
        # create the metadata
        self.fileName = fileName
        self.samp_rate = samp_rate
        self.freq = freq
        self.author = author
        self.description = description
        self.metadataIsSet = True
    
    def createMetadata(self):
        self.metadata = SigMFFile(
            data_file=self.fileName+'.sigmf-data', # extension is optional
            global_info = {
                SigMFFile.DATATYPE_KEY: get_data_type_str(self.data),  # in this case, 'cf32_le'
                SigMFFile.SAMPLE_RATE_KEY: self.samp_rate,
                SigMFFile.AUTHOR_KEY: self.author,
                SigMFFile.DESCRIPTION_KEY: self.description,
                SigMFFile.FREQUENCY_KEY: self.freq,
                SigMFFile.DATETIME_KEY: self.date_time,
            }
        )
        self.metadata.tofile(self.fileName+'.sigmf-meta')

In [4]:


def APILink(IP,port,path):
    url = "http://"+IP+":"+port+path 
    print("API URL:",url)
    return url 

def recordIQ(interfaceIP,IP,port,samples):
    path = "/rx/recordIQ"
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.get(url, headers=headers)
    response_json = response.json()
    imag = response_json["imag"]
    real = response_json["real"]
    return real,imag

def setRxIQ(interfaceIP,IP,port):
    path = "/rx/set/IQ"
    data = {
        "contents": "IQ"
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.get(url, data=json.dumps(data), headers=headers)
    return response
    
def setRxMPSK(interfaceIP,IP,port,M):
    path = "/rx/set/MPSK"
    data = {
        "M": M
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response

def setPHY(interfaceIP,IP,port,params):
    path = "/set/PHY"
    data = {
        "x": params["x"],
        "freq": params["freq"],
        "SamplingRate": params["SamplingRate"],
        "gain": params["gain"]
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response
    
def set_tx_sinusoid(interfaceIP,IP,port):
    path = "/tx/set/sinusoid"
    data = {
        "message": "sinusoid"
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response

def set_tx_MPSK(interfaceIP,IP,port,M):
    path = "/tx/set/MPSK"
    data = {
        "M": M
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response

def set_tx_pnSequence(interfaceIP,IP,port,sequence):
    path = "/tx/set/pnSequence"
    data = {
        "sequence": sequence
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response

def set_tx_fileSource(interfaceIP,IP,port,fileSource):
    path = "/tx/set/fileSource"
    data = {
        "fileSource": fileSource
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response

def start_tx(interfaceIP,IP,port):
    path = "/tx/start"
    data = {
        "message": "TX Start"
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response
    
def stop_tx(interfaceIP,IP,port):
    path = "/tx/stop"
    data = {
        "message": "sinusoid"
    }
    headers = {'Content-Type': 'application/json'}
    url = APILink(IP,port,path)
    updateSocket(interfaceIP)
    # session = setSession(IP,interfaceIP)
    response = requests.post(url, data=json.dumps(data), headers=headers)
    return response

def plotTimeDomain(I,Q,samples=-1,id=0):
    plt.plot(I[0:samples], color='red')
    plt.plot(Q[0:samples], color='blue')
    plt.xlabel('Time')
    plt.ylabel('IQ')
    plt.title('Time Domain Plot Node: '+str(id))
    plt.grid(True)
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.show()

def plotConstellationDiagram(I,Q,samples=-1,id=0):
    plt.scatter(I[0:samples], Q[0:samples], color='red')
    plt.xlabel('I')
    plt.ylabel('Q')
    plt.title('Constellation Diagram Plot Node: '+str(id))
    plt.grid(True)
    plt.axhline(0, color='black',linewidth=0.5)
    plt.axvline(0, color='black',linewidth=0.5)
    plt.show()

def plotSpectrogram(I,Q,fs,samples=-1,id=0):
    x = np.array([complex(Q[i],I[i]) for i in range(len(I))])
    f, t, Sxx = signal.spectrogram(x, fs)
    plt.title('Spectrogram Plot Node: '+str(id))
    plt.pcolormesh(t, f, Sxx, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

In [5]:
def RoundRecordData(sessions,params,type,tx,RX,samples=1024,packets=1, metadata = {}):
    print("type:",type)
    txInterface = next((sessions[sessionType]["interface"] for sessionType in sessions if tx in sessions[sessionType]["nodes"]), None)
    print("TX Interface",txInterface)
    if type == "sinusoid":
        response = set_tx_sinusoid(txInterface,NodeIP[tx],port["radio"])
    elif type == "MPSK":
        response = set_tx_MPSK(txInterface,NodeIP[tx],port["radio"],metadata[type])
    elif type == "pnSequence":
        response = set_tx_pnSequence(txInterface,NodeIP[tx],port["radio"],metadata[type])
    elif type == "fileSource":
        print("Setting file source")
        response = set_tx_fileSource(txInterface,NodeIP[tx],port["radio"],metadata[type])
    response = setPHY(txInterface,NodeIP[tx],port["radio"],params["tx"])
    response = start_tx(txInterface,NodeIP[tx],port["radio"])
    time.sleep(0.1)
    data = {}
    for rx in RX:
        data[rx] = np.array([])
        rxInterface =  next((sessions[sessionType]["interface"] for sessionType in sessions if rx in sessions[sessionType]["nodes"]), None)
        print("RX Interface",rxInterface)
        response = setRxIQ(rxInterface,NodeIP[rx],port["radio"])
        response = setPHY(rxInterface,NodeIP[rx],port["radio"],params["rx"])
    for i in range(packets):
        #print("packet "+str(i)+" of "+str(packets))
        for rx in RX:
            rxInterface =  next((sessions[sessionType]["interface"] for sessionType in sessions if rx in sessions[sessionType]["nodes"]), None)
            print("RX Interface",rxInterface)
            I,Q = recordIQ(rxInterface,NodeIP[rx],port["radio"],samples)
            complexIQ = np.array(I)+np.array(Q)*1j
            data[rx] = np.append(data[rx],complexIQ)
            plotTimeDomain(I,Q)
            #plotConstellationDiagram(I,Q)
            #plotSpectrogram(I,Q)
            #plotAvgPower(I,Q)
    time.sleep(0.1)
    response = stop_tx(txInterface,NodeIP[tx],port["radio"])
    return data

def RecordDataAmbient(params,RX,samples=1024):
    for rx in RX:
        print("start RX ",rx)
        response = setRxIQ(NodeIP[rx],port["radio"])
        print(response)
        I,Q = recordIQ(NodeIP[rx],port["radio"],samples)
        print("stop RX",rx)
        plotTimeDomain(I,Q,samples=1024)
        plotConstellationDiagram(I,Q,samples=1024)
    #return data
    
def formatData(Data,modulation):
    DS_node = np.array([])
    DS_labels = np.array([])
    DS_data = np.array([])
    for node in Data:
        for N in range(examples):
            DS_node = np.append(DS_node,node)
            DS_labels = np.append(DS_labels,modulation)
            complex_data = np.array(Data[node][N*samplesPerExample:(N+1)*samplesPerExample])
            imag_data = complex_data.imag
            real_data = complex_data.real
            real_imag_data = np.array([[real_data,  imag_data]])
            if len(DS_data) == 0: 
                DS_data = real_imag_data
            else:
                DS_data = np.append(DS_data,real_imag_data,axis=0)
            #print(DS_data.shape)

    return DS_data, DS_node, DS_labels

def recordDataSignalClassificationMs(TXs,RXs,params,Ms = [2,4]):
    for tx in TXs:
        env = {"rx":RXs,"tx":tx}
        Ms = [2,4]
        for M in Ms:
            Data = RoundRecordData(params,type="MPSK",tx=env["tx"],RX=env["rx"],
                                   samples=samplesPerExample,packets=int(packetsNeeded),
                                   metadata={"MPSK":M})
            if M == 2:
                Modulation = "BPSK"
            elif M == 4:
                Modulation = "QPSK"    
            DS_data, DS_node, DS_labels = formatData(Data,Modulation)
            rxText = "_".join(str(rx) for rx in env["rx"])
            filename = Modulation+"_"+str(env["tx"])+"_TX_"+rxText+"_RX_20042024"
            file = h5py.File(filename+".h5", "w")
            # Create datasets within the HDF5 file
            file.create_dataset("data", data=DS_data)
            file.create_dataset("node", data=DS_node)
            file.close()
            
def recordDataSignalClassification(sessions,TXs,RXs,params,modulations,packetsNeeded):
    datetime = dt.datetime.now().strftime("%Y%m%d")
    for tx in TXs:
        env = {"rx":RXs,"tx":tx}
        for modulation in modulations:
            print("Recording data for "+modulation+" TX=" +str(tx))
            fileSource = "Matlab/"+modulation+".dat"
            Data = RoundRecordData(sessions,params,type="fileSource",tx=env["tx"],RX=env["rx"],
                                   samples=samplesPerExample,packets=int(packetsNeeded),
                                   metadata={"fileSource":fileSource})   
            DS_data, DS_node, DS_labels = formatData(Data,modulation)
            rxText = "_".join(str(rx) for rx in env["rx"])
            filename = modulation+"_"+str(env["tx"])+"_TX_"+rxText+"_RX_"+datetime
            file = h5py.File("data/"+filename+".h5", "w")
            # Create datasets within the HDF5 file
            file.create_dataset("data", data=DS_data)
            file.create_dataset("node", data=DS_node)
            file.close()

    #Save json file containing all parameters for experiment
    json_object = json.dumps(params, indent=4)
    JSON_FILE_NAME = "Parameters_TX_"+"".join([str(tx)+"_" for tx in TXs])+"RX_"+"".join([str(rx)+"_" for rx in RXs])+"_"+str(len(modulations))+"Modulations_"+datetime+".json"
    with open(JSON_FILE_NAME, "w") as jsonFile:
        jsonFile.write(json_object)
            
def recordDataChannelFingerprinting(TXs,RXs,params,sequence = "glfsr"):
    for tx in TXs:
        print("TX:",tx)
        env = {"rx":RXs,"tx":tx}
        Data = RoundRecordData(params,type="pnSequence",tx=env["tx"],RX=env["rx"],samples=samplesPerExample,packets=int(packetsNeeded),M=1,sequence=sequence)
        DS_data, DS_node, DS_labels = formatData(Data,sequence)
        rxText = "_".join(str(rx) for rx in env["rx"])
        filename = "pnSequence"+"_"+str(env["tx"])+"_TX_"+rxText+"_RX_20042024"
        file = h5py.File(filename+".h5", "w")
        # Create datasets within the HDF5 file
        file.create_dataset("data", data=DS_data)
        file.create_dataset("node", data=DS_node)
        file.close()

In [13]:
class BindAdapter(HTTPAdapter):
    def __init__(self, source_ip, **kwargs):
        self.source_ip = source_ip
        super().__init__(**kwargs)

    def init_poolmanager(self, *args, **kwargs):
        kwargs['source_address'] = (self.source_ip, 0)
        super().init_poolmanager(*args, **kwargs)

    def build_response(self, req, resp):
        return super().build_response(req, resp)

def setSession(hostIP,IP):
    # Create separate sessions for each interface
    # session = requests.Session()
    # session.mount("http://", BindAdapter(IP))
    # session.mount("https://", BindAdapter(IP)) 
    print(hostIP)
    print(IP)
    sock = socket.create_connection((hostIP, 80), source_address=(IP, 0))
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter()
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def updateSocket(IP):
    old_socket = socket.socket
    def new_socket(*args, **kwargs):
        sock = old_socket(*args, **kwargs)
        sock.bind((IP, 0))
        return sock
    socket.socket = new_socket

def getInterfaces():
    myInterfaces = {"WIFI":{"name":"en0","address":None},"ETH":{"name":"en17","address":None}}

    interfaces = psutil.net_if_addrs()
    for interface, addresses in interfaces.items():
        for interfaceName in myInterfaces:
            if interface == myInterfaces[interfaceName]["name"]:
                for addr in addresses:
                    if addr.family.name == "AF_INET":
                        myInterfaces[interfaceName]["address"] = addr.address
                        print(f"For {interfaceName} IP is {myInterfaces[interfaceName]['address']}")
    return myInterfaces

In [ ]:
NodeIP = {1:"10.15.7.109",2:"10.13.251.58",3:"10.15.6.201",4:"10.15.7.67",5:"10.15.7.121",
      6:"10.15.6.180",7:"10.15.7.12",8:"10.15.7.52",9:"127.0.0.1",10:"127.0.0.1"}

port = {'orch':'5001','radio':'5002','ai':'5003'}

examples= 10000
samplesPerExample = 1024
samplesPerPacket = 8192
freq = 2.29e9
samp_rate = 6e5
gainRX = 60
gainTX = 0
Total_samples = examples * samplesPerExample 
print("Total_samples:",Total_samples)
examplesPerPacket = samplesPerPacket/samplesPerExample   #I_Sig,Q_Sig
print("examplesPerPacket:",examplesPerPacket)
packetsNeeded = int(math.ceil(examples/examplesPerPacket))
print("packetsNeeded:",packetsNeeded)
paramsTx = {"x":"tx","freq":freq,"SamplingRate":samp_rate,"gain":gainTX}
paramsRx = {"x":"rx","freq":paramsTx["freq"],"SamplingRate":int(paramsTx["SamplingRate"]*2),"gain":gainRX}
params = {"tx":paramsTx,"rx":paramsRx}

# modulations = ["BPSK", "QPSK", "8PSK", "16QAM", "64QAM", "PAM4", "GFSK", "CPFSK", "B-FM", "DSB-AM", "SSB-AM"]
modulations = ["BPSK"]

# Define the source IPs for each network interface
myInterfaces = getInterfaces()
interfaceSessions = {"wifi": {"interface":myInterfaces["WIFI"]["address"],"nodes":[2]}, "eth":{"interface":myInterfaces["ETH"]["address"],"nodes":[1]}}

TXs = [1]
RXs = [2]

#recordDataSignalClassificationMs(TXs,RXs,params,Ms = [2,4])
recordDataSignalClassification(interfaceSessions,TXs,RXs,params,modulations = modulations, packetsNeeded=packetsNeeded)


Total_samples: 10240000
examplesPerPacket: 8.0
packetsNeeded: 1250
Recording data for BPSK TX=1
type: fileSource
TX Interface 10.15.7.37
Setting file source
API URL: http://10.15.7.109:5002/tx/set/fileSource
API URL: http://10.15.7.109:5002/set/PHY


ConnectionError: HTTPConnectionPool(host='10.15.7.109', port=5002): Max retries exceeded with url: /set/PHY (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x137dc5040>: Failed to establish a new connection: [Errno 22] Invalid argument'))